<a href="https://colab.research.google.com/github/gitmystuff/DTSC5810/blob/main/Week_05-Distributions/Week_05_Survival_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Week 05 - Survival Analysis

Survival analysis is a branch of statistics for analyzing the expected duration of time until an event occurs, such as death in biological organisms or failure in mechanical systems. This topic is also called reliability theory or reliability analysis in engineering, duration analysis or duration modelling in economics, and event history analysis in sociology. Survival analysis attempts to answer certain questions, such as what is the proportion of a population which will survive past a certain time? Of those that survive, at what rate will they die or fail? Can multiple causes of death or failure be taken into account? How do particular circumstances or characteristics increase or decrease the probability of survival?

https://en.wikipedia.org/wiki/Survival_analysis

$S(t) = P(T>t)$

where
* T is the random lifetime taken from the population and it cannot be negative
* S(t) value lies between 0 and 1 and it is a non-increasing function

In [1]:
# pip install lifelines

## Readings

* https://scikit-survival.readthedocs.io/en/stable/user_guide/index.html
* https://www.kaggle.com/code/saychakra/survival-analysis-of-lung-cancer-patients
* https://medium.com/@zynp.atlii/survival-analysis-simplified-explaining-and-applying-with-python-7efacf86ba32
* https://www.kdnuggets.com/2020/07/complete-guide-survival-analysis-python-part1.html
* https://towardsdatascience.com/survival-analysis-intuition-implementation-in-python-504fde4fcf8e
* https://towardsdatascience.com/hands-on-survival-analysis-with-python-270fa1e6fb41
* https://github.com/Idilismiguzel/Machine-Learning/blob/master/Survival_Analysis/Survival_Analysis.ipynb
* https://www.kdnuggets.com/2020/07/complete-guide-survival-analysis-python-part1.html
* https://medium.com/the-researchers-guide/survival-analysis-in-python-km-estimate-cox-ph-and-aft-model-5533843c5d5d
* https://stat.ethz.ch/R-manual/R-devel/library/survival/html/lung.html

## Key Concepts

* Time to Event: Relapse, Improvement, Death
* Stratified sampling to deal with volume of data (equal or near equal amount of subjects from groups)
* Survival Curves
* Kaplan Meier Estimator (Product Limit Estimator)
* Censoring
* Log Rank Test

## Uses

* Cancer Studies
* Event History Analysis
* Failure Analysis - when will a battery fail
* Traffic - What happens after the light turns red
* Churn and Attrition
* Retention Rate
* Processes Reaching Critical Values
* Time From Sale Contact to Sale
* Warranty Claims

In [2]:
# # make the dataframe
# import pandas as pd

# lung = pd.read_csv('https://raw.githubusercontent.com/gitmystuff/Datasets/main/lung.csv')
# lung.head()

## Info

* status: dead or alive
* time: length of observation
* inst: number in bucket

for example 3 reached event 2 within 306 units of time

In [3]:
# # check nulls and data types
# lung.info()

In [4]:
# # checkout shapes
# import matplotlib.pyplot as plt

# lung.hist()
# plt.tight_layout();

In [5]:
# # what's the index of a missing value?
# lung[lung['inst'].isnull()].index.tolist()

In [6]:
# # the values of observation 155
# lung.loc[155]

In [7]:
# # compare missing observation with other like data
# lung[(lung['sex'] == 1) & (lung['status'] == 2) & (lung['time'] > 300) & (lung['time'] < 400)]

In [8]:
# # get descriptive stats for inst
# lung[(lung['sex'] == 1) & (lung['status'] == 2) & (lung['time'] > 300) & (lung['time'] < 400)].describe()

In [9]:
# int(lung['inst'].where((lung['sex'] == 1) & (lung['status'] == 2) & (lung['time'] > 300) & (lung['time'] < 400)).mean())

In [10]:
# # replace missing value with mean
# lung['inst'].fillna(int(lung['inst'].where((lung['sex'] == 1) & (lung['status'] == 2) & (lung['time'] > 300) & (lung['time'] < 400)).mean()), inplace=True)
# lung.info()

In [11]:
# # get index of ph.ecog missing value
# lung[lung['ph.ecog'].isnull()].index.tolist()

In [12]:
# # get info
# lung.loc[13]

In [13]:
# lung[(lung['sex'] == 1) & (lung['status'] == 2) & (lung['age'] == 60)]

In [14]:
# lung['ph.ecog'].fillna(1.0, inplace=True)
# lung.info()

In [15]:
# # get index of ph.karno missing value
# lung[lung['ph.karno'].isnull()].index.tolist()

In [16]:
# # get info
# lung.loc[205]

In [17]:
# lung[(lung['sex'] == 1) & (lung['status'] == 2) & (lung['ph.ecog'] == 2.0) & (lung['pat.karno'] == 70)]

In [18]:
# lung[(lung['sex'] == 1) & (lung['status'] == 2) & (lung['ph.ecog'] == 2.0) & (lung['pat.karno'] == 70)].describe()

In [19]:
# lung['ph.karno'].fillna(60.0, inplace=True)
# lung.info()

In [20]:
# # get index of ph.karno missing value
# lung[lung['pat.karno'].isnull()].index.tolist()

In [21]:
# # get info
# lung.loc[[66, 78, 104]]

In [22]:
# # lung.grouby('pat.karno')['sex'].mean()
# lung.groupby(['pat.karno', 'sex'])['ph.karno'].mean()

In [23]:
# lung.loc[104,'pat.karno'] = 70
# lung.loc[66,'pat.karno'] = 80
# lung.loc[78,'pat.karno'] = 90
# lung.info()

In [24]:
# lung[['meal.cal', 'wt.loss']].hist();

In [25]:
# lung['meal.cal'].fillna(lung['meal.cal'].mode()[0], inplace=True)
# lung['wt.loss'].fillna(lung['wt.loss'].median(), inplace=True)
# lung.info()

## Kaplan Meier Estimator

The Kaplan–Meier estimator, also known as the product limit estimator, is a non-parametric statistic used to estimate the survival function from lifetime data.

https://en.wikipedia.org/wiki/Kaplan%E2%80%93Meier_estimator

Kaplan-Meier is the most commonly used life-table method in medical practice. It adequately copes with the issues such as, such as patients for whom the event has not yet occurred and for those lost to follow up. The data required by the method include the time of commencement of the treatment and the time the measured event occurred (e.g., death, relapse, or hospitalization). Patients who dropped out from treatment and those who are still alive at the end of the study period are “censored”, but the contribution that they have made to the event probability are fully accounted for.

https://advancedrenaleducation.com/wparep/article/life-table-survival-analysis/

Fraction of observations who survived for a certain amount of time under the same circumstances.

$\hat{S}(t) = \prod(1 - \frac{d_i}{n_i})$

where
* t is time
* d is number of events
* n is number of individuals

In [26]:
# # train the estimator
# from lifelines import KaplanMeierFitter

# model = KaplanMeierFitter()
# model.fit(durations = lung.time, event_observed = lung.status)
# model.event_table

## Survival Curves

In [27]:
# # survival curve
# import matplotlib.pyplot as plt

# model.plot()
# plt.title("Kaplan-Meier")
# plt.xlabel("Days")
# plt.ylabel("Survival");

In [28]:
# # the S curve; https://smallbusiness.chron.com/s-curve-business-23032.html
# import numpy as np
# import matplotlib.pyplot as plt
# import scipy.stats as stats

# x = np.linspace(-3, 3, 100)
# y = stats.norm.cdf(x)
# plt.plot(x, y, 'b', label='cdf')
# plt.title('The S Curve')
# plt.xlabel('x')
# plt.ylabel('cdf(x)')
# plt.grid(True)
# plt.show();

In [29]:
# # create models for males and females
# m = lung[lung['sex']==1]
# male_model = KaplanMeierFitter()
# male_model.fit(durations = m.time, event_observed = m.status)
# f = lung[lung['sex']==2]
# female_model = KaplanMeierFitter()
# female_model.fit(durations = f.time, event_observed = f.status)

In [30]:
# # plot males and females
# male_model.plot(label = 'male')
# female_model.plot(label = 'female')
# plt.title("Kaplan-Meier by Sex")
# plt.xlabel("Days")
# plt.ylabel("Survival")

## Censoring

https://towardsdatascience.com/introduction-to-survival-analysis-6f7e19c31d96

* Right Censoring
* Left Censoring
* Interval Censoring

Censoring is a form of missing data problem in which time to event is not observed for reasons such as termination of study before all recruited subjects have shown the event of interest or the subject has left the study prior to experiencing an event. Censoring is common in survival analysis.

**Right censoring** will occur, for example, for those subjects whose birth date is known but who are still alive when they are lost to follow-up or when the study ends. We generally encounter right-censored data. If the event of interest has already happened before the subject is included in the study but it is not known when it occurred, the data is said to be **left-censored**. When it can only be said that the event happened between two observations or examinations, this is **interval censoring**.

Left censoring occurs for example when a permanent tooth has already emerged prior to the start of a dental study that aims to estimate its emergence distribution. In the same study, an emergence time is interval-censored when the permanent tooth is present in the mouth at the current examination but not yet at the previous examination.

https://en.wikipedia.org/wiki/Survival_analysis#Censoring

## Log Rank Test

* Compares survival probabilities of two groups
* Null hypothesis: there is no difference
* Hazard Ratio: a statistical measure that compares the likelihood of an event occurring in one group to another over time and should be constant throughout the study
* May not be appropriate if survival curves cross

The logrank test, or log-rank test, is a hypothesis test to compare the survival distributions of two samples. It is a nonparametric test and appropriate to use when the data are right skewed and censored (technically, the censoring must be non-informative).

https://en.wikipedia.org/wiki/Logrank_test

The logrank test is used to test the null hypothesis that there is no difference between the populations in the probability of an event (here a death) at any time point. The analysis is based on the times of events (here deaths)... Because the logrank test is purely a test of significance it cannot provide an estimate of the size of the difference between the groups or a confidence interval. For these we must make some assumptions about the data. Common methods use the hazard ratio

https://www.ncbi.nlm.nih.gov/pmc/articles/PMC403858/

Numbers at risk in the first and second groups in the sixth month were 70 and 30 with 10 deaths. We would expect 7 to come from group 1 and 3 from group 2.

* $10 * 70 / (70 + 30) = 7$
* $10 * 30 / (70 + 30) = 3$

Log Rank Test basically indicates whether survival between two groups is signicantly different or not.

In [31]:
# # log rank test
# from lifelines.statistics import logrank_test

# logrank_test(durations_A=m.time,
#              durations_B=f.time,
#              event_observed_A=m.status,
#              event_observed_B=f.status)

p is not low supporting the null hypothesis that the two groups are similar

## Cox Hazard Regression

Proportional hazards models are a class of survival models in statistics. Survival models relate the time that passes, before some event occurs, to one or more covariates that may be associated with that quantity of time... There is a relationship between proportional hazards models and Poisson regression models which is sometimes used to fit approximate proportional hazards models in software for Poisson regression.

https://en.wikipedia.org/wiki/Proportional_hazards_model

* The survival function is a function of time and shows the proportion of subject's status at time t
* Cox Hazard Regression lets us see the impact of all variants on the status
* Adjustments are made based on variance

In [32]:
# # engineer the status for better performance /  true false

# lung.loc[lung.status == 1, "status"] = 0
# lung.loc[lung.status == 2, "status"] = 1
# lung.drop('inst', axis=1, inplace=True)

In [33]:
# from lifelines import CoxPHFitter

# model = CoxPHFitter()
# model.fit(lung, 'time', event_col = 'status')
# model.print_summary()

In [34]:
# # plot the model
# model.plot()

* This plot shows that sex and ph.ecog are significant (p values of .01 and <.005 respectively)
* Logs: how many times to multiply a number to get to another number - https://www.mathsisfun.com/algebra/logarithms.html
* The log of 2^3 is three

## Widgets

* https://g.co/gemini/share/d3c41e86f380